In [1]:
!pip install singlestoredb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.4/357.4 kB 850.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 2.4 MB/s eta 0:00:00


In [11]:
import singlestoredb as s2
from faker import Faker
import json
from datetime import datetime, timedelta
import random
import os

In [10]:

# Initialize Faker
fake = Faker()

# Connection parameters
DB_HOST = 'svc-3482219c-a389-4079-b18b-d50662524e8a-shared-dml.aws-virginia-6.svc.singlestore.com'
DB_PORT = 3333
DB_USER = 'david-2e12e'
DB_PASSWORD = 'Heheda3120@'
DB_NAME = 'db_david_5328b'
SSL_CERT_PATH = 'singlestore_bundle.pem'  # Assuming the file is in the same directory as the notebook

# Ensure the SSL certificate file exists
if not os.path.exists(SSL_CERT_PATH):
    raise FileNotFoundError(f"SSL certificate file not found at {SSL_CERT_PATH}")

# Read the SSL certificate
with open(SSL_CERT_PATH, 'rb') as cert_file:
    ssl_cert = cert_file.read()

# Connect to SingleStore
conn = s2.connect(
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME,
    ssl={'ca': ssl_cert}
)


KeyboardInterrupt: 

In [ ]:

# Create tables
with conn.cursor() as cur:
    # Users table
    cur.execute('''
    CREATE TABLE IF NOT EXISTS Users (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        email VARCHAR(255) NOT NULL,
        password VARCHAR(255) NOT NULL,
        createdAt TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')

    # UserProfile table
    cur.execute('''
    CREATE TABLE IF NOT EXISTS UserProfile (
        id INT AUTO_INCREMENT PRIMARY KEY,
        userId INT NOT NULL,
        desiredJobTitle VARCHAR(255),
        jobType ENUM('Full-Time', 'Part-Time', 'Freelance'),
        preferredIndustry JSON,
        experienceYears INT,
        education TEXT,
        createdAt TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updatedAt TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
        FOREIGN KEY (userId) REFERENCES Users(id)
    )
    ''')

    # WorkExperience table
    cur.execute('''
    CREATE TABLE IF NOT EXISTS WorkExperience (
        id INT AUTO_INCREMENT PRIMARY KEY,
        userId INT NOT NULL,
        companyName VARCHAR(255) NOT NULL,
        jobTitle VARCHAR(255) NOT NULL,
        startDate DATE,
        endDate DATE,
        responsibilities TEXT,
        achievements TEXT,
        FOREIGN KEY (userId) REFERENCES Users(id)
    )
    ''')

    # Skills table
    cur.execute('''
    CREATE TABLE IF NOT EXISTS Skills (
        id INT AUTO_INCREMENT PRIMARY KEY,
        workExperienceId INT NOT NULL,
        skillName VARCHAR(255) NOT NULL,
        proficiencyLevel INT,
        FOREIGN KEY (workExperienceId) REFERENCES WorkExperience(id)
    )
    ''')

    # CareerGoals table
    cur.execute('''
    CREATE TABLE IF NOT EXISTS CareerGoals (
        id INT AUTO_INCREMENT PRIMARY KEY,
        userId INT NOT NULL,
        shortTermGoals TEXT,
        longTermGoals TEXT,
        targetDate DATE,
        FOREIGN KEY (userId) REFERENCES Users(id)
    )
    ''')

# Function to generate synthetic data
def generate_synthetic_data(num_users=50):
    for _ in range(num_users):
        # Generate User
        with conn.cursor() as cur:
            cur.execute(
                'INSERT INTO Users (name, email, password) VALUES (%s, %s, %s)',
                (fake.name(), fake.email(), fake.password())
            )
            user_id = cur.lastrowid

        # Generate UserProfile
        with conn.cursor() as cur:
            cur.execute(
                'INSERT INTO UserProfile (userId, desiredJobTitle, jobType, preferredIndustry, experienceYears, education) VALUES (%s, %s, %s, %s, %s, %s)',
                (
                    user_id,
                    fake.job(),
                    random.choice(['Full-Time', 'Part-Time', 'Freelance']),
                    json.dumps([fake.job(), fake.job()]),
                    random.randint(0, 20),
                    fake.paragraph()
                )
            )

        # Generate Work Experiences
        num_experiences = random.randint(1, 5)
        for _ in range(num_experiences):
            with conn.cursor() as cur:
                start_date = fake.date_between(start_date='-10y', end_date='today')
                end_date = fake.date_between(start_date=start_date, end_date='today')
                cur.execute(
                    'INSERT INTO WorkExperience (userId, companyName, jobTitle, startDate, endDate, responsibilities, achievements) VALUES (%s, %s, %s, %s, %s, %s, %s)',
                    (
                        user_id,
                        fake.company(),
                        fake.job(),
                        start_date,
                        end_date,
                        fake.paragraph(),
                        fake.paragraph()
                    )
                )
                exp_id = cur.lastrowid

            # Generate Skills for each Work Experience
            num_skills = random.randint(2, 6)
            for _ in range(num_skills):
                with conn.cursor() as cur:
                    cur.execute(
                        'INSERT INTO Skills (workExperienceId, skillName, proficiencyLevel) VALUES (%s, %s, %s)',
                        (
                            exp_id,
                            fake.job_skill(),
                            random.randint(1, 5)
                        )
                    )

        # Generate Career Goals
        with conn.cursor() as cur:
            cur.execute(
                'INSERT INTO CareerGoals (userId, shortTermGoals, longTermGoals, targetDate) VALUES (%s, %s, %s, %s)',
                (
                    user_id,
                    fake.paragraph(),
                    fake.paragraph(),
                    fake.date_between(start_date='today', end_date='+5y')
                )
            )

# Generate synthetic data
generate_synthetic_data(50)

print("Synthetic data generation complete!")

# Close the connection
conn.close()